# **P8 - Kriging Geográfico (2D)**

*En esta parte de la práctica, implementaremos un modelo simple de Kriging sobre los datos de <br>
calidad del aire en Madrid, proporcionados por el Ayuntamiento en su [portal
de datos abiertos](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=9e42c176313eb410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default).*

Como de costumbre, primero importamos las librerías y módulos necesarios.

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from geopy import Nominatim, Photon
from PIL import Image
from pyproj import Transformer
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel, RationalQuadratic, ExpSineSquared

# descomenta %matplotlib qt si prefieres plots interactivos
%matplotlib inline
# %matplotlib qt

## 0. Setup y Funciones Auxiliares

### 0.1 Transformación de coordenadas

Las coordenadas geográficas que emplearemos vienen dadas en grados (*latitud* y *longitud*). <br>
Por tanto, necesitamos una forma de transformarlas (o *proyectarlas*) a unidades métricas (p. ej. metros), <br>
por ser éstas más adecuadas en el cálculo de distancias necesarias en **Kriging**.

Para estas transformaciones, usaremos la librería **PyProj** [[*GH*](https://github.com/pyproj4/pyproj) | [*Docs*](https://pyproj4.github.io/pyproj/stable/)].

<div class="alert alert-info">
&#9432; <strong>INFO</strong> <br>
<em> Estas transformaciones se realizan usando los siguientes
<strong>sistemas de referencia de coordenadas</strong> (<strong>CRS</strong>): <br>
</em>

* <em>EPSG:<strong>4326</strong></em> (WGS84) <br>
por ser el CRS apropiado para coordenadas de latitud y longitud (representadas en grados)
[<a href="https://epsg.io/?q=4326">link</a>].

* <em>EPSG:<strong>32630</strong></em> (UTM, zona 30) <br>
CRS empleado para expresar en metros las coordenadas, proyectándolas a un plano 2D local. <br>
La zona 30 es la correspondiente a Zaragoza [<a href="https://www.ign.es/web/coordenadas-de-estaciones-ergnss">link</a>].

</div>



In [6]:
transformer_longlat_xy = Transformer.from_crs(4326, 32630, always_xy=True)


def from_longlat_to_xy(long, lat):
    """Transforma coordenadas de longitud y latitud a coordenadas x, y en CRS 32630
    (UTM, zona 30)

    Args:
        long: (n,) array o float con longitud (en grados)
        lat: (n,) array o float con latitud (en grados)

    Returns:
        x: (n,) array o float con coordenadas x en CRS 32630
        y: (n,) array o float con coordenadas y en CRS 32630
    """
    return transformer_longlat_xy.transform(long, lat)


def from_xy_to_longlat(x, y):
    """Transforma coordenadas x, y en CRS 32630 (UTM, zona 30) a coordenadas de longitud
    y latitud

    Args:
        x: (n,) array o float con coordenadas x en CRS 32630
        y: (n,) array o float con coordenadas y en CRS 32630

    Returns:
        long: (n,) array o float con longitud (en grados)
        lat: (n,) array o float con latitud (en grados)
    """
    return transformer_longlat_xy.transform(x, y, direction="inverse")

### 0.2 Carga de datos

En primer lugar, definimos  las **coordenadas geográficas** (latitud y longitud) <br>
**de las estaciones** de medición de calidad del aire en Madrid (ver [enlace]((https://datos.madrid.es/FWProjects/egob/Catalogo/MedioAmbiente/Aire/Ficheros/Interprete_ficheros_%20calidad_%20del_%20aire_global.pdf)), Anexo 1).


In [7]:
# Coordinates for the stations
station_coordinates = {
    "04": [-3.7122567, 40.4238823],
    "08": [-3.6823158, 40.4215533],
    "11": [-3.6773491, 40.4514734],
    "16": [-3.6392422, 40.4400457],
    "17": [-3.7133167, 40.347147],
    "18": [-3.7318356, 40.3947825],
    "24": [-3.7473445, 40.4193577],
    "27": [-3.5800258, 40.4769179],
    "35": [-3.7031662, 40.4192091],
    "36": [-3.6453104, 40.4079517],
    "38": [-3.7071303, 40.4455439],
    "39": [-3.7115364, 40.4782322],
    "40": [-3.6515286, 40.3881478],
    "47": [-3.6868138, 40.3980991],
    "48": [-3.6903729, 40.4398904],
    "49": [-3.6824999, 40.4144444],
    "50": [-3.6887449, 40.4655841],
    "54": [-3.6121394, 40.3730118],
    "55": [-3.5805649, 40.4623628],
    "56": [-3.7187679, 40.3850336],
    "57": [-3.6605173, 40.4942012],
    "58": [-3.7746101, 40.5180701],
    "59": [-3.609031, 40.465144],
    "60": [-3.6897308, 40.5005477]
}

# Create the new dictionary with the required structure
data = {
    "estacion": list(station_coordinates.keys()),
    "latitud": [item[1] for item in station_coordinates.values()],
    "longitud": [item[0] for item in station_coordinates.values()]
}

df_estaciones = pd.DataFrame(data)


Ahora, añadimos las **coordenadas métricas** $x, y$ (en metros) correspondientes al *CRS UTM (zona 30)* de cada estación:

In [8]:
df_estaciones[["x", "y"]] = df_estaciones.apply(
    lambda row: from_longlat_to_xy(row.longitud, row.latitud),
    axis=1,
    result_type="expand",
)
df_estaciones

,estacion,latitud,longitud,x,y
0,04,40.423882,-3.712257,439579.332691,4.475049e+06
1,08,40.421553,-3.682316,442117.239817,4.474771e+06
2,11,40.451473,-3.677349,442564.048969,4.478089e+06
3,16,40.440046,-3.639242,445786.171919,4.476796e+06
4,17,40.347147,-3.713317,439420.701759,4.466532e+06
5,18,40.394782,-3.731836,437891.695775,4.471833e+06
6,24,40.419358,-3.747345,436598.563517,4.474572e+06
7,27,40.476918,-3.580026,450835.202403,4.480854e+06
8,35,40.419209,-3.703166,440346.357207,4.474524e+06
9,36,40.407952,-3.645310,445245.509000,4.473237e+06


Por último, cargamos el fichero `historicoXX.csv` correspondiente a la calidad del aire medida en cada estación.

In [10]:
path_calidad = "historicoNOx.csv"
df_aire = pd.read_csv(path_calidad, sep=",")
df_aire


# Convert the 'date' column to datetime
df_aire['date'] = pd.to_datetime(df_aire['date'])

# Define the start and end date for the range
# You can change this to define how you average the data
start_date = '2024-01-01'
end_date = '2024-12-31'

# Filter the DataFrame between the two dates
filtered_df = df_aire[(df_aire['date'] >= start_date) & (df_aire['date'] <= end_date)]

# Calculate the average for each station between the two dates
average_values = filtered_df.drop('date', axis=1).mean()

# Display the average values for each station
print(filtered_df)
print(average_values)



           date     04     08     11    16     17     18    24     27     35  \
8399 2024-01-01   51.0   53.0   55.0  59.0   70.0   57.0  27.0   69.0   47.0   
8400 2024-01-02  107.0   82.0   95.0  80.0  117.0  122.0  75.0  115.0   96.0   
8401 2024-01-03   40.0   52.0   52.0  56.0   54.0   45.0  17.0   84.0   43.0   
8402 2024-01-04   53.0   51.0   61.0  55.0   54.0   51.0  23.0   80.0   50.0   
8403 2024-01-05   24.0   28.0   29.0  18.0   27.0   18.0   5.0   17.0   27.0   
...         ...    ...    ...    ...   ...    ...    ...   ...    ...    ...   
8729 2024-11-26   96.0  102.0  100.0  77.0  172.0   91.0  51.0   99.0  100.0   
8730 2024-11-27  125.0   82.0  101.0  60.0  235.0  114.0  61.0  109.0   98.0   
8731 2024-11-28  136.0  100.0  132.0  87.0  282.0  118.0  65.0  105.0  117.0   
8732 2024-11-29  145.0  125.0  114.0  75.0  243.0  135.0  90.0   95.0  123.0   
8733 2024-11-30  117.0   97.0  103.0  72.0  249.0  107.0  58.0   99.0   91.0   

      ...    48    49     50     54    

### 0.3 Visualizador interactivo

La siguiente clase está basada en [**Plotly**](https://plotly.com/graphing-libraries/)
(principalmente su [API de visualización de mapas](https://plotly.com/python/mapbox-layers/)), <br>
y define métodos auxiliares para visualizar los resultados de la práctica. <br>

In [11]:
class MapPlotter:
    """Visualizador interactivo de mapas con Plotly"""

    CENTER = {"lat": 41.65573, "lon": -0.88616}

    def __init__(self, zoom=12, center=None, showlegend=False, **kwargs):
        self.fig = go.Figure()
        self.fig.update_layout(
            mapbox_style="open-street-map",  # o p. ej. "carto-positron",
            mapbox_zoom=zoom,
            mapbox_center=center or self.CENTER,
            margin={"r": 0, "t": 10, "l": 0, "b": 10},
            showlegend=showlegend,
            # autosize=True,
            **kwargs,
        )

    def add_longlat_points(
        self,
        lon,
        lat,
        s=15,
        color=None,
        text=None,
        edgecolor=None,
        name=None,
        adjust_center=True,
        **kwargs,
    ):
        if edgecolor is not None:
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=lat,
                    lon=lon,
                    mode="markers",
                    marker_size=1.1 * s,
                    marker_color=edgecolor,
                    showlegend=False,
                    **kwargs,
                )
            )

        self.fig.add_trace(
            go.Scattermapbox(
                lat=lat,
                lon=lon,
                mode="markers",
                marker_size=s,
                marker_color=color,
                text=text,
                name=name,
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_density_heatmap(self, lon, lat, z, radius=10, adjust_center=True, **kwargs):
        self.fig.add_trace(
            go.Densitymapbox(
                lat=lat.ravel(),
                lon=lon.ravel(),
                z=z.ravel(),
                radius=radius,
                colorbar_title="μg/m³",
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_raster_heatmap(
        self,
        xgrid,
        ygrid,
        heatmap,
        cmap="Spectral_r",
        adjust_center=True,
        add_cbar=True,
        **kwargs,
    ):
        # rasterize the heatmap
        heatmap_ = (heatmap - heatmap.min()) / np.ptp(heatmap)
        heatmap_ = plt.get_cmap(cmap)(heatmap_)[..., :3]
        raster_hm = Image.fromarray((heatmap_ * 255).clip(0, 255).astype(np.uint8))

        # coordenadas de las esquinas del heatmap
        bbox = np.array(
            [
                [xgrid[0, 0], ygrid[0, 0]],  # top-left
                [xgrid[0, -1], ygrid[0, -1]],  # top-right
                [xgrid[-1, -1], ygrid[-1, -1]],  # bottom-right
                [xgrid[-1, 0], ygrid[-1, 0]],  # bottom-left
            ]
        )
        lon, lat = from_xy_to_longlat(bbox[:, 0], bbox[:, 1])
        coords = np.stack((lon, lat), axis=1)

        layers = [*self.fig.layout.mapbox.layers]  # type: ignore
        layers.append(
            {
                "sourcetype": "image",
                "source": raster_hm,
                "coordinates": coords,
                "below": "traces",
                **kwargs,
            }
        )
        self.fig.layout.mapbox.layers = layers  # type: ignore

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

        if add_cbar:
            # add colorbar via invisible scatter trace
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=[None],
                    lon=[None],
                    mode="markers",
                    marker=dict(
                        colorscale=cmap,
                        color=heatmap,
                        cmin=heatmap.min(),
                        cmax=heatmap.max(),
                        showscale=True,
                        colorbar=dict(
                            title="μg/m³",
                            # titleside="right",
                            ticks="outside",
                        ),
                    ),
                )
            )

    def adjust_center(self, lon, lat):
        self.fig.update_layout(mapbox_center={"lat": lat, "lon": lon})

    def show(self):
        self.fig.show()

## 1. Kriging Geográfico

Con el setup anterior, en esta parte deberéis implementar el modelo de **Kriging** para interpolar <br>
la calidad del aire en Madrid, y visualizar los resultados obtenidos.

Para ello os proponemos utilizar, y completar en su caso, las siguientes funciones:

<div class="alert alert-danger">
&#9432; <strong>WARNING</strong> <br>
<em> Algunas estaciones no tienen todos los datos. Los archivos proporcionados son los que tienen datos. No obstante, al seleccionar las fechas podéis encontrar periodos sin datos que os dará un error.
</em>


</div>

In [12]:
def create_regular_kriging_grid(X, npts=100):
    """Malla de coordenadas sobre las que estimar la calidad del aire mediante kriging

    Args:
        X: (..., 2) array con las coordenadas xy de las observaciones
        npts: número de puntos a lo largo y ancho de la malla

    Returns:
        xgrid: (npts, npts) array con las coordenadas x de la malla
        ygrid: (npts, npts) array con las coordenadas y de la malla
    """
    # límites de la malla
    x_min, x_max = X[..., 0].min(), X[..., 0].max()
    y_min, y_max = X[..., 1].min(), X[..., 1].max()
    # malla regular
    x, y = np.linspace(x_min, x_max, npts), np.linspace(y_min, y_max, npts)
    xgrid, ygrid = np.meshgrid(x, y)
    return xgrid, ygrid


def air_quality_kriging(df_aire, gpr, npts_grid=100, log_data=False):
    """Estimación de la calidad del aire en Zaragoza mediante kriging

    Args:
        df_aire: DataFrame con los datos de calidad del aire
        gpr: Proceso Gaussiano a entrenar.
        npts_grid: int, número de puntos a lo largo y ancho de la malla de coordenadas
            sobre las que estimar la calidad del aire
        log_data: si True, imprime las observaciones del contaminante en cada estación

    Returns:
        (xgrid, ygrid): tupla con las coordenadas, de shape (npts, npts) cada una, x e y
            de la malla.
        concentracion_pred: (npts, npts) array con la concentración media estimada del
            contaminante en cada punto (coordenada) de la malla.
        sigma: (npts, npts) array con la desviación estándar de la concentración estimada
            del contaminante en cada punto (coordenada) de la malla.
    """
    # Como observaciones, usaremos la media de los valores de contaminante por estación
    # a lo largo del año 2023:
    # añade las observaciones a su estacion correspondiente en un dataframe auxiliar
    df_est = df_estaciones.copy()
    df_est["concen."] = df_est["estacion"].map(average_values)

    # Drop missing values (NaN)
    df_est = df_est.dropna(axis=0, how='any')
    assert (
        df_est["concen."].notnull().all()
    ), "Faltan observaciones para algunas estaciones"
    print(df_est)

    if log_data:
        print(df_est)

    # Datos de entrada para el proceso Gaussiano
    X = df_est[["x", "y"]].values
    concentraciones_observadas = df_est["concen."].values

    # creación de una malla regular sobre la que estimar la calidad del aire.
    xgrid, ygrid = create_regular_kriging_grid(X, npts=npts_grid)

    # entrenamiento del Proceso Gaussiano con las observaciones
    # TODO: tu código


    # copia y reshape de la malla para ajustarla a la shape que espera scikit-learn
    # TODO: tu código


    # predicción de la concentración y su desviación estándar
    # TODO: tu código


    # reshape de las predicciones para ajustarlas a la malla: shape (npts, npts)
    # TODO: tu código


    return (xgrid, ygrid), concentracion_pred, sigma

*Definición del proceso Gaussiano e interpolación de la calidad del aire:*

En primer lugar probamos con RBF:

In [ ]:
# Utiliza un GausianProcess y la función air quality kriging
# gpr = GaussianProcessRegressor(...)
# malla, concent_pred, concent_sigma = air_quality_kriging(...)




fig = MapPlotter()

# visualización de las estaciones en el mapa como puntos:
fig.add_longlat_points(
    df_estaciones["longitud"],
    df_estaciones["latitud"],
     text=df_estaciones.estacion,
    color="white",
    edgecolor="black",
)

# visualización de la concentración predicha con kriging:
fig.add_raster_heatmap(malla[0], malla[1], concent_pred, opacity=0.6)

# llamada final para mostrar el mapa
fig.show()

Vamos a probar con otro kernel, RationalQuadratic:

In [1]:
# Utiliza un GausianProcess y la función air quality kriging
# gpr = GaussianProcessRegressor(...)
# malla, concent_pred, concent_sigma = air_quality_kriging(...)


Parece que estamos obteniendo resultados similares a RBF.